In [1]:
import pandas as pd

In [6]:
# Load CSVs
customers = pd.read_csv("C:/Airflow/dags/data/df_customers.csv")
orders = pd.read_csv("C:/Airflow/dags/data/df_orders.csv")
order_items = pd.read_csv("C:/Airflow/dags/data/df_order_items.csv")
payments = pd.read_csv("C:/Airflow/dags/data/df_payments.csv")
products = pd.read_csv("C:/Airflow/dags/data/df_products.csv")

# Print structure
print(customers.shape, customers.head() , customers.isnull().sum())
print(orders.shape, orders.head(), orders.isnull().sum())
print(order_items.shape, order_items.head(),order_items.isnull().sum())
print(payments.shape, payments.head(),payments.isnull().sum())
print(products.shape, products.head(),products.isnull().sum())


(89316, 4)     customer_id  customer_zip_code_prefix       customer_city customer_state
0  hCT0x9JiGXBQ                     58125     varzea paulista             SP
1  PxA7fv9spyhx                      3112  armacao dos buzios             RJ
2  g3nXeJkGI0Qw                      4119             jandira             SP
3  EOEsCQ6QlpIg                     18212          uberlandia             MG
4  mVz5LO2Vd6cL                     88868            ilhabela             SP customer_id                 0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
(89316, 7)        order_id   customer_id order_status order_purchase_timestamp  \
0  Axfy13Hk4PIk  hCT0x9JiGXBQ    delivered      2017-10-22 18:57:54   
1  v6px92oS8cLG  PxA7fv9spyhx    delivered      2018-06-20 21:40:31   
2  Ulpf9skrhjfm  g3nXeJkGI0Qw    delivered      2018-02-16 16:19:31   
3  bwJVWupf2keN  EOEsCQ6QlpIg    delivered      2018-08-18 18:04:29   
4  Dd0QnrMk9Cj5  mVz5LO2Vd6c

# Cleaning Orders File 

In [8]:

orders = pd.read_csv("C:/Airflow/dags/data/df_orders.csv", parse_dates=[
    'order_purchase_timestamp', 'order_approved_at', 
    'order_delivered_timestamp', 'order_estimated_delivery_date'
])

# Clean
orders = orders.dropna(subset=['order_delivered_timestamp'])
orders['order_status'] = orders['order_status'].str.lower()

#Create a derived column
orders['delivery_time_days'] = (
    orders['order_delivered_timestamp'] - orders['order_purchase_timestamp']
).dt.days

# Preview
orders.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,delivery_time_days
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,4
1,v6px92oS8cLG,PxA7fv9spyhx,delivered,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24,13
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,delivered,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08,10
3,bwJVWupf2keN,EOEsCQ6QlpIg,delivered,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19,9
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,delivered,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18,14


In [28]:
orders=orders.dropna(subset=['order_approved_at'])

In [29]:
#save cleaned file 
orders.to_csv("C:/Airflow/dags/cleaned_orders.csv", index=False)

# Cleaning Orders Item File

In [12]:
order_items['total_price'] = order_items['price']+ order_items['shipping_charges']
order_items.head()

,order_id,product_id,seller_id,price,shipping_charges,total_price
0,Axfy13Hk4PIk,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65,308.16
1,v6px92oS8cLG,qejhpMGGVcsl,IjlpYfhUbRQs,170.80,23.79,194.59
2,Ulpf9skrhjfm,qUS5d2pEAyxJ,77p2EYxcM9MD,64.40,17.38,81.78
3,bwJVWupf2keN,639iGvMyv0De,jWzS0ayv9TGf,264.50,30.72,295.22
4,Dd0QnrMk9Cj5,1lycYGcsic2F,l1pYW6GBnPMr,779.90,30.66,810.56


In [17]:
#save 
order_items.to_csv('C:/Airflow/dags/cleaned_order_items.csv',index=False)

# Cleaning Payment File

In [15]:
payments.drop(columns=['payment_sequential'], inplace=True)


In [18]:
payments.to_csv('C:/Airflow/dags/cleaned_payments.csv',index=False)

# Cleaning Product File

In [21]:
#Drop missing values from columns
products = products.dropna(subset=[
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
])

#fill missing values with unknow
products['product_category_name'] = products['product_category_name'].fillna('unknown')
#lowercase the values
products['product_category_name'] = products['product_category_name'].str.lower().str.strip()

# Create new derived column: product volume (in cm³)
products['product_volume_cm3'] = (
    products['product_length_cm'] *
    products['product_height_cm'] *
    products['product_width_cm'])
#save
products.to_csv("C:/Airflow/dags/cleaned_products.csv", index=False)

In [23]:
products.values.tolist()

[['90K0C1fIyQUf', 'toys', 491.0, 19.0, 12.0, 16.0, 3648.0],
 ['qejhpMGGVcsl', 'watches_gifts', 440.0, 18.0, 14.0, 17.0, 4284.0],
 ['qUS5d2pEAyxJ',
  'costruction_tools_garden',
  2200.0,
  16.0,
  16.0,
  16.0,
  4096.0],
 ['639iGvMyv0De', 'toys', 1450.0, 68.0, 3.0, 48.0, 9792.0],
 ['1lycYGcsic2F', 'toys', 300.0, 17.0, 4.0, 12.0, 816.0],
 ['TpFZIk33uyGn', 'toys', 2800.0, 16.0, 31.0, 39.0, 19344.0],
 ['Y0T4Ca4BXK8U', 'bed_bath_table', 3975.0, 46.0, 12.0, 43.0, 23736.0],
 ['IUykT5z5LcPW', 'toys', 250.0, 45.0, 15.0, 35.0, 23625.0],
 ['0UHKKyYmxWfq', 'toys', 1200.0, 30.0, 23.0, 30.0, 20700.0],
 ['MJ82dNJNiV7U', 'toys', 1500.0, 30.0, 5.0, 20.0, 3000.0],
 ['wdV7yJzaz3Su', 'toys', 4338.0, 17.0, 12.0, 14.0, 2856.0],
 ['V3ksL8RRt2uO', 'toys', 350.0, 21.0, 22.0, 22.0, 10164.0],
 ['vndXhZ66Bwpl', 'toys', 500.0, 16.0, 12.0, 22.0, 4224.0],
 ['2RxPIb6Sjlm1', 'toys', 300.0, 38.0, 19.0, 22.0, 15884.0],
 ['DXG0TFaZjGHO', 'toys', 200.0, 16.0, 7.0, 11.0, 1232.0],
 ['YCMS6jUWbEbe', 'toys', 4000.0, 45.0, 2